**Machine Learning Assignment 4 (CNN, AdaBoost, Bagging)**

**Author: Jaemin Lee**

**Due: 12/2/2019**

Importing a library that is not in google colab

In [0]:
# install pytorch on google colab
!pip install -q matplotlib-venn

In [0]:
!apt-get -qq install -y libfluidsynth1

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


In [0]:
# import libraries
import sklearn
import numpy as np
import pandas as pd

In [0]:
# google drive import 
# setting up google drive connection
from google.colab import drive
drive.mount('/content/gdrive')

Getting the data

In [0]:
df = pd.read_csv('gdrive/My Drive/mnist.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Task 1: Subsampling 100 instances from each label

In [0]:
# create an empty data frame
res = pd.DataFrame([])

# run the for loop in the range of 10 
for i in range(10):
  tempFrame = df.loc[df.label == i, :]               # labels from 0 to 9
  frame = tempFrame.sample(n=100, random_state=0)    # extract 100 random samples from each label
  res = res.append(pd.DataFrame(frame))              # combine all 100 instances from each label

print(res)

      label  pixel0  pixel1  pixel2  ...  pixel780  pixel781  pixel782  pixel783
9818      0       0       0       0  ...         0         0         0         0
8643      0       0       0       0  ...         0         0         0         0
2986      0       0       0       0  ...         0         0         0         0
5681      0       0       0       0  ...         0         0         0         0
6818      0       0       0       0  ...         0         0         0         0
...     ...     ...     ...     ...  ...       ...       ...       ...       ...
5118      9       0       0       0  ...         0         0         0         0
5845      9       0       0       0  ...         0         0         0         0
1128      9       0       0       0  ...         0         0         0         0
3883      9       0       0       0  ...         0         0         0         0
6499      9       0       0       0  ...         0         0         0         0

[1000 rows x 785 columns]


In [0]:
# separate X and y variables to split test and train data
X = res.iloc[:,1:] # all rows and columns except the labels
y = res.iloc[:, 0] # all rows and just the labels

Train and Test Split

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =1)

Task 2: Using PyTorch to build CNN

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

Getting built-in dataset for CNN

In [0]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7*7*32, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.softmax = nn.Softmax()
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out

In [0]:
model = ConvNet(num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/5], Step [100/600], Loss: 1.5745
Epoch [1/5], Step [200/600], Loss: 1.5478
Epoch [1/5], Step [300/600], Loss: 1.4922
Epoch [1/5], Step [400/600], Loss: 1.5089
Epoch [1/5], Step [500/600], Loss: 1.4941
Epoch [1/5], Step [600/600], Loss: 1.4878
Epoch [2/5], Step [100/600], Loss: 1.4877
Epoch [2/5], Step [200/600], Loss: 1.4807
Epoch [2/5], Step [300/600], Loss: 1.4711
Epoch [2/5], Step [400/600], Loss: 1.4738
Epoch [2/5], Step [500/600], Loss: 1.5004
Epoch [2/5], Step [600/600], Loss: 1.4807
Epoch [3/5], Step [100/600], Loss: 1.4865
Epoch [3/5], Step [200/600], Loss: 1.4737
Epoch [3/5], Step [300/600], Loss: 1.4844
Epoch [3/5], Step [400/600], Loss: 1.4714
Epoch [3/5], Step [500/600], Loss: 1.4710
Epoch [3/5], Step [600/600], Loss: 1.4908
Epoch [4/5], Step [100/600], Loss: 1.4690
Epoch [4/5], Step [200/600], Loss: 1.4914
Epoch [4/5], Step [300/600], Loss: 1.4825
Epoch [4/5], Step [400/600], Loss: 1.4711
Epoch [4/5], Step [500/600], Loss: 1.4620
Epoch [4/5], Step [600/600], Loss:

In [0]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy of the model on the 10000 test images: 98.68 %


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Task 3: AdaBoost and Bagging

In [0]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

adaboost_dtree = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=1))
adaboost_dtree_model = adaboost_dtree.fit(X_train, y_train)


adaboost_dtree_pred = adaboost_dtree.predict(X_test)
print("Accuracy for AdaBoost is with Decision Tree is:",metrics.accuracy_score(y_test, adaboost_dtree_pred)*100, "%")

Accuracy for AdaBoost is with Decision Tree is: 60.5 %


In [0]:
adaboost_logistic = AdaBoostClassifier(LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))
adaboost_log_model = adaboost_logistic.fit(X_train, y_train)


adaboost_log_pred = adaboost_logistic.predict(X_test)
print("Accuracy for AdaBoost with Logistic Regression is:",metrics.accuracy_score(y_test, adaboost_log_pred)*100,"%")

Accuracy for AdaBoost with Logistic Regression is: 82.0 %


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
bagging_dtree = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
bagging_dtree_model = bagging_dtree.fit(X_train, y_train)


bagging_dtree_pred = bagging_dtree.predict(X_test)
print("Accuracy for Bagging with Decision Tree is:",metrics.accuracy_score(y_test, bagging_dtree_pred)*100,"%")

Accuracy for Bagging with Decision Tree is: 79.5 %


In [0]:
bagging_log = BaggingClassifier(LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))
bagging_log_model = bagging_log.fit(X_train, y_train)


bagging_log_pred = bagging_log.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

In [0]:
print("Accuracy for Bagging with Logistic Regression is:",metrics.accuracy_score(y_test, bagging_log_pred)*100,"%")

Accuracy for Bagging with Logistic Regression is: 83.0 %


Task 4: Precision and Recall

In [0]:
from sklearn.metrics import recall_score

recall_1 = recall_score(y_test, adaboost_dtree_pred, average = 'macro')*100
recall_2 = recall_score(y_test, adaboost_log_pred, average = 'macro')*100
recall_3 = recall_score(y_test, bagging_dtree_pred, average = 'macro')*100
recall_4 = recall_score(y_test, bagging_log_pred, average = 'macro')*100

In [0]:
print("Recall for Adaboost with Decision Tree: ",recall_1,"%")
print("Recall for Adaboost with Logistic Regression: ",recall_2,"%")
print("Recall for Bagging with Decision Tree: ",recall_3,"%")
print("Recall for Bagging with Logistic Regression: ",recall_4,"%")

Recall for Adaboost with Decision Tree:  59.81556420875307 %
Recall for Adaboost with Logistic Regression:  81.81087952295384 %
Recall for Adaboost with Decision Tree:  77.47922201637371 %
Recall for Adaboost with Logistic Regression:  83.16367481847358 %


In [0]:
from sklearn.metrics import precision_score

p_1 = precision_score(y_test, adaboost_dtree_pred, average = 'macro')*100
p_2 = precision_score(y_test, adaboost_log_pred, average = 'macro')*100
p_3 = precision_score(y_test, bagging_dtree_pred, average = 'macro')*100
p_4 = precision_score(y_test, bagging_log_pred, average = 'macro')*100

In [0]:
print("Precision for Adaboost with Decision Tree: ", p_1,"%")
print("Precision for Adaboost with Logistic Regression: ",p_2,"%")
print("Precision for Bagging with Decision Tree: ", p_3,"%")
print("Precision for Bagging with Logistic Regression: ", p_4,"%")

Precision for Adaboost with Decision Tree:  60.21801950062821 %
Precision for Adaboost with Logistic Regression:  81.30774564870676 %
Precision for Bagging with Decision Tree:  79.5959595959596 %
Precision for Bagging with Logistic Regression:  82.52597402597402 %
